In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [1]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [ ]:

imgs = np.load('75/images.npy')
imgs = imgs.astype('float32')
indices = np.random.permutation(imgs.shape[0])
imgs = imgs[indices]

split = int(18000*0.7)

train_imgs = imgs[100:split]
test_imgs = imgs[split:]
val_imgs = imgs[0:100]

train_imgs = train_imgs / 255.0
test_imgs = test_imgs / 255.0
val_imgs = val_imgs / 255.0

labels = np.load('75/labels.npy')
labels = labels.astype('int32')
labels = labels[indices]
train_labels = labels[100:split]
test_labels = labels[split:]
val_labels = labels[0:100]

train_imgs = train_imgs.reshape((train_imgs.shape[0], 75, 75, 1))
val_imgs = val_imgs.reshape((val_imgs.shape[0], 75, 75, 1))
test_imgs = test_imgs.reshape((test_imgs.shape[0], 75, 75, 1))

## Classification Model

In [70]:

def conv_time(time):
    ntime = 0
    if time[1] > 30:
        ntime = (time[0] + 0.5)
    else:
        ntime = time[0]
    return ntime

train_labels_converted = np.array([conv_time(time) for time in train_labels])
test_labels_converted = np.array([conv_time(time) for time in test_labels])
val_labels_converted = np.array([conv_time(time) for time in val_labels])

encoder = LabelEncoder()
test_labels_encoded = encoder.fit_transform(test_labels_converted.reshape(-1))
train_labels_encoded = encoder.fit_transform(train_labels_converted.reshape(-1))
val_labels_encoded = encoder.fit_transform(val_labels_converted.reshape(-1))

OHencoder = OneHotEncoder(sparse_output=False)
train_labels_oh = OHencoder.fit_transform(train_labels_encoded.reshape(-1, 1))
val_labels_oh = OHencoder.fit_transform(val_labels_encoded.reshape(-1, 1))


In [71]:
input_shape = (75, 75, 1)
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(kernel_size=(5,5), strides = (2,2), activation="relu", filters=32))
model.add(keras.layers.Conv2D(activation="relu", filters=32, kernel_size=(3,3), input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=512, activation="relu"))
model.add(keras.layers.Dense(units=512, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dense(units=32, activation="relu"))
model.add(keras.layers.Dense(units=24, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

print(train_imgs.shape, train_labels_oh.shape)
val_labels_oh = val_labels_oh.reshape((val_labels_oh.shape[0], 24))

model.fit(train_imgs, train_labels_oh, epochs=10, batch_size=256, validation_data=(val_imgs, val_labels_oh), callbacks=[early_stop])

(12500, 75, 75, 1) (12500, 24)
Epoch 1/10
49/49 [==============================] - 7s 106ms/step - loss: 3.1781 - accuracy: 0.0438 - val_loss: 3.1790 - val_accuracy: 0.0200
Epoch 2/10
49/49 [==============================] - 5s 103ms/step - loss: 3.1779 - accuracy: 0.0445 - val_loss: 3.1801 - val_accuracy: 0.0200
Epoch 3/10
49/49 [==============================] - 5s 104ms/step - loss: 3.1781 - accuracy: 0.0445 - val_loss: 3.1788 - val_accuracy: 0.0200
Epoch 4/10
49/49 [==============================] - 5s 105ms/step - loss: 3.1779 - accuracy: 0.0445 - val_loss: 3.1795 - val_accuracy: 0.0200
Epoch 5/10
49/49 [==============================] - 5s 105ms/step - loss: 3.1778 - accuracy: 0.0445 - val_loss: 3.1803 - val_accuracy: 0.0200
Epoch 6/10
49/49 [==============================] - 5s 108ms/step - loss: 3.1777 - accuracy: 0.0445 - val_loss: 3.1802 - val_accuracy: 0.0200
Epoch 7/10
49/49 [==============================] - 6s 126ms/step - loss: 3.1777 - accuracy: 0.0445 - val_loss: 3.179

In [76]:
preds = model.predict(test_imgs)
preds = np.argmax(preds, axis=1)

results = encoder.inverse_transform(preds)

accuracy = np.sum(results == test_labels_converted) / len(test_labels_converted)
print(accuracy*100, '%')

169/169 [==============================] - 1s 6ms/step
4.018518518518518 %


## Regression Model

In [1]:
ttm_reg = keras.models.Sequential()
ttm_reg.add(keras.layers.Conv2D(activation='relu', filters=32, kernel_size=(3,3), strides = (2,2),input_shape=input_shape))
ttm_reg.add(keras.layers.MaxPooling2D(pool_size=2))
ttm_reg.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
ttm_reg.add(keras.layers.Conv2D(filters=32 ,kernel_size=(3,3), activation='relu'))
ttm_reg.add(keras.layers.MaxPooling2D(pool_size=2))
ttm_reg.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
ttm_reg.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))


ttm_reg.add(keras.layers.Flatten())
ttm_reg.add(keras.layers.Dense(units=256, activation='relu'))
ttm_reg.add(keras.layers.Dense(units=256, activation='relu'))
ttm_reg.add(keras.layers.Dense(units=256, activation='relu'))
ttm_reg.add(keras.layers.Dense(units=256, activation='relu'))
ttm_reg.add(keras.layers.Dropout(0.1))
ttm_reg.add(keras.layers.Dense(units=128, activation='relu'))
ttm_reg.add(keras.layers.Dense(units=64, activation='relu'))
ttm_reg.add(keras.layers.Dense(units=1, activation="softplus"))
early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)
ttm_reg.compile(loss='mse', optimizer="adam", metrics=['mae'])
ttm_reg.fit(train_imgs, y_train_ttm_reg, epochs=40, batch_size = 512, validation_data = (valid_images,y_valid_ttm_reg), callbacks = [early_stop])

NameError: name 'keras' is not defined